In [1]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle

/opt/conda/lib/python3.8/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
with open('lumi.json') as f:
    lumis = json.load(f)

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [3]:
year = '2017'
infiles = subprocess.getoutput("ls infiles/"+year+"*.json").split()
coffeadir_prefix = 'outfiles-plots/'

In [4]:
repickle = True
# Check if pickle exists, don't recreate it if it does
picklename = year+'/cutflow.pkl'
#if os.path.isfile(picklename):
#    repickle = False

In [5]:
if repickle:
    outsum = processor.dict_accumulator()

    for this_file in infiles:
    
        print(this_file)
        index = this_file.split("_")[1].split(".json")[0]
        filename = coffeadir_prefix+year+"_dask_"+index+".coffea"
        
        with open(this_file, 'r') as openfile:
            samples = json.load(openfile)

        print(filename)
        if os.path.isfile(filename):
            out = util.load(filename)

            if len(outsum.keys()) == 0:
                outsum['cutflow'] = out['cutflow']
                outsum['sumw'] = out['sumw']
            else:
                outsum['cutflow'].add(out['cutflow'])
                outsum['sumw'].add(out['sumw'])
                
            del out
        else:
            print('Missing file '+index,index)
            #print("File " + filename + " is missing")  
            
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
    outsum['cutflow'].scale(scale_lumi, 'dataset')
    cutflow = outsum['cutflow'].group('dataset', hist.Cat('process', 'Process'), pmap)
    
    del outsum
    
    outfile = open(picklename, 'wb')
    pickle.dump(cutflow, outfile, protocol=-1)
    outfile.close()

infiles/2017_DYJetsToLL.json
outfiles-plots/2017_dask_DYJetsToLL.coffea
infiles/2017_Diboson.json
outfiles-plots/2017_dask_Diboson.coffea
infiles/2017_EWKV.json
outfiles-plots/2017_dask_EWKV.coffea
infiles/2017_GluGluHToBB.json
outfiles-plots/2017_dask_GluGluHToBB.coffea
infiles/2017_JetHTData.json
outfiles-plots/2017_dask_JetHTData.coffea
infiles/2017_QCD.json
outfiles-plots/2017_dask_QCD.coffea
infiles/2017_SingleMuData.json
outfiles-plots/2017_dask_SingleMuData.coffea
infiles/2017_SingleTop.json
outfiles-plots/2017_dask_SingleTop.coffea
infiles/2017_TTbar.json
outfiles-plots/2017_dask_TTbar.coffea
infiles/2017_VBFHToBB.json
outfiles-plots/2017_dask_VBFHToBB.coffea
infiles/2017_WHToBB.json
outfiles-plots/2017_dask_WHToBB.coffea
infiles/2017_WJetsToLNu.json
outfiles-plots/2017_dask_WJetsToLNu.coffea
infiles/2017_WJetsToQQ.json
outfiles-plots/2017_dask_WJetsToQQ.coffea
infiles/2017_ZHToBB.json
outfiles-plots/2017_dask_ZHToBB.coffea
infiles/2017_ZJetsToQQ.json
outfiles-plots/2017_dask_Z

/opt/conda/lib/python3.8/site-packages/coffea/hist/hist_tools.py:376: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7f09bc96a760>
  warnings.warn(


In [6]:
# Read the histogram from the pickle file
templates = pickle.load(open(picklename,'rb'))

In [7]:
cutflow = templates.sum('genflavor').integrate('region',['signal-ggf'])
cutflow

<Hist (process,cut) instance at 0x7f09bc36fdf0>

In [8]:
cutflow.values()

{('QCD',): array([5.40033825e+08, 5.40033825e+08, 5.40033825e+08, 5.39038996e+08,
        1.60109523e+07, 1.60109523e+07, 4.27875398e+06, 3.93209118e+06,
        3.88469115e+06, 3.71362248e+06, 3.56759967e+06, 1.12301220e+04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('ttbar',): array([1.50805091e+07, 1.50805091e+07, 1.50805091e+07, 1.50690247e+07,
        1.46668932e+05, 1.46668932e+05, 5.42001116e+04, 2.88777612e+04,
        2.23826675e+04, 1.35518173e+04, 1.31542076e+04, 1.59489170e+02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('singlet',): array([1.42983919e+06, 1.42983919e+06, 1.42983919e+06, 1.42871175e+06,
        1.17135541e+04, 1.17135541e+04, 5.17665526e+03, 2.90499523e+03,
        2.18875543e+03, 1.47961864e+03, 1.45038764e+03, 1.65731662e+01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('Wjets',): array([1.17735180e+08, 1.17735180e+08, 1.17735180e+08, 1.17671985e+08,
        3.00089153e+05, 3.00089153e+05, 1.20476388e+05, 1.

In [9]:
df1 = pd.DataFrame([])

df1['ggF'] = cutflow.values()[('ggF',)]
df1['VBF'] = cutflow.values()[('VBF',)]
df1['WH'] = cutflow.values()[('WH',)]
df1['ZH'] = cutflow.values()[('ZH',)]
df1['ttH'] = cutflow.values()[('ttH',)]


df1
df1 = df1[:-3].astype('int')
df1
df1.index = ['nothing','trigger','lumimask','metfilter','minjetkin','jetid','n2ddt','antiak4btagMediumOppHem','met','noleptons','notvbf','ddbpass']
df1

,ggF,VBF,WH,ZH,ttH
nothing,2204,13335,11389,4646,6630
trigger,2204,13335,11389,4646,6630
lumimask,2204,13335,11389,4646,6630
metfilter,2200,13324,11382,4642,6622
minjetkin,285,117,110,79,301
jetid,285,117,110,79,301
n2ddt,142,65,56,41,109
antiak4btagMediumOppHem,121,56,44,27,37
met,116,53,32,20,29
noleptons,111,50,27,17,17


In [10]:
df1 = df1[4:-2]
df1.index = ['Jet kinematics','Jet ID','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons']
df1.to_latex(buf=year+'/cutflow-sig.tex')
df1

/tmp/ipykernel_28133/2958735745.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df1.to_latex(buf=year+'/cutflow-sig.tex')


,ggF,VBF,WH,ZH,ttH
Jet kinematics,285,117,110,79,301
Jet ID,285,117,110,79,301
Jet $N_2^\text{DDT}$,142,65,56,41,109
Opp. hem. b veto,121,56,44,27,37
MET $<$ 140 GeV,116,53,32,20,29
No leptons,111,50,27,17,17


In [11]:
df2 = pd.DataFrame([])

df2['QCD'] = cutflow.values()[('QCD',)]
df2['Wjets'] = cutflow.values()[('Wjets',)]
df2['Zjets'] = cutflow.values()[('Zjets',)]
df2['EWKW'] = cutflow.values()[('EWKW',)]
df2['EWKZ'] = cutflow.values()[('EWKZ',)]
df2['VV'] = cutflow.values()[('VV',)]
df2['ttbar'] = cutflow.values()[('ttbar',)]
df2['singlet'] = cutflow.values()[('singlet',)]

df2 = df2[:-3].astype('int')
df2.index = ['nothing','trigger','lumimask','metfilter','minjetkin','jetid','n2ddt','antiak4btagMediumOppHem','met','noleptons','notvbf','ddbpass']
df2

,QCD,Wjets,Zjets,EWKW,EWKZ,VV,ttbar,singlet
nothing,540033824,117735180,19005358,2302677,283360,2002246,15080509,1429839
trigger,540033824,117735180,19005358,2302677,283360,2002246,15080509,1429839
lumimask,540033824,117735180,19005358,2302677,283360,2002246,15080509,1429839
metfilter,539038996,117671984,18989488,2300531,283026,2001088,15069024,1428711
minjetkin,16010952,300089,134439,23764,3141,4855,146668,11713
jetid,16010952,300089,134439,23764,3141,4855,146668,11713
n2ddt,4278753,120476,60719,12473,1695,2682,54200,5176
antiak4btagMediumOppHem,3932091,110464,54520,9325,1485,2416,28877,2904
met,3884691,104150,53292,6276,1430,1856,22382,2188
noleptons,3713622,94913,38657,5258,959,1376,13551,1479


In [12]:
df2 = df2[4:-2]
df2.index = ['Jet kinematics','Jet ID','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons']
df2.to_latex(buf=year+'/cutflow-bkg.tex')
df2

/tmp/ipykernel_28133/2909133722.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df2.to_latex(buf=year+'/cutflow-bkg.tex')


,QCD,Wjets,Zjets,EWKW,EWKZ,VV,ttbar,singlet
Jet kinematics,16010952,300089,134439,23764,3141,4855,146668,11713
Jet ID,16010952,300089,134439,23764,3141,4855,146668,11713
Jet $N_2^\text{DDT}$,4278753,120476,60719,12473,1695,2682,54200,5176
Opp. hem. b veto,3932091,110464,54520,9325,1485,2416,28877,2904
MET $<$ 140 GeV,3884691,104150,53292,6276,1430,1856,22382,2188
No leptons,3713622,94913,38657,5258,959,1376,13551,1479
